In [10]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, MetaData, Table, Column, ForeignKey,Integer

In [11]:
# Create Database Connection
engine = create_engine('sqlite:///../wine_reviews.sqlite')
Base = automap_base()

In [12]:
Base.prepare(engine, reflect=True)
Base.classes.keys()

['reviews']

In [13]:
reviews=Base.classes.reviews
# point=Base.classes.prpt
session=Session(engine)

In [14]:
first_row=session.query(reviews).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x133101d12e8>,
 'country': 'US',
 'description': 'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.',
 'id': 1,
 'latitude': 45.3442868,
 'longitude': -122.6670374,
 'points': 87,
 'price': 14,
 'province': 'Oregon',
 'region_1': 'Willamette Valley',
 'title': 'Rainstorm 2013 Pinot Gris (Willamette Valley)',
 'variety': 'Pinot Gris',
 'winery': 'Rainstorm'}

In [15]:
titles=session.query(reviews.variety).distinct().count()
titles

256

In [16]:
state=[]
variety=[]
points=[]
price=[]
title_cnt=[]

for row in session.query(reviews.province,reviews.variety,func.avg(reviews.points).label('Average_Point'),\
                         func.avg(reviews.price).label('Average_Price'),func.count(reviews.title).label('Title_cnt')).group_by(reviews.province,\
                        reviews.variety).order_by(reviews.province).all():
    state.append(row.province)
    variety.append(row.variety)
    title_cnt.append(row.Title_cnt)
    points.append(round(row.Average_Point,2))
    if (row.Average_Price != None):
        price.append(round(row.Average_Price))
    else:
        price.append(0)


In [17]:
state_prpt=pd.DataFrame({"State":state,
                    "Variety":variety,
                    "Avg_Points":points,
                    "Avg_Price":price,
                    "Title_Count":title_cnt}, columns=['State','Variety','Avg_Points','Avg_Price','Title_Count'])

state_prpt.head()

,State,Variety,Avg_Points,Avg_Price,Title_Count
0,Arizona,Chardonnay,84.67,30,3
1,Arizona,Malvasia,85.00,28,1
2,Arizona,Malvasia Bianca,85.00,21,1
3,Arizona,Mourvèdre,84.00,36,2
4,Arizona,Petite Sirah,86.50,50,2


In [18]:
engine = create_engine('sqlite:///../wine_reviews.sqlite')
state_prpt.to_sql('prpt', engine, if_exists='replace')

In [10]:
df=pd.read_sql('prpt',engine)
df.head()
st='Ohio'

In [11]:
df[(df['State']==st)]

,index,State,Variety,Avg_Points,Avg_Price,Title_Count
450,450,Ohio,Cabernet Franc,83.00,15,1
451,451,Ohio,Cabernet Sauvignon,84.00,12,1
452,452,Ohio,Chardonnay,85.50,14,2
453,453,Ohio,Nebbiolo,80.00,15,1
454,454,Ohio,Riesling,84.43,14,7


In [13]:
df[(df['State']==st)].to_json(orient='records')
# to_json(orient='records')

'[{"index":450,"State":"Ohio","Variety":"Cabernet Franc","Avg_Points":83.0,"Avg_Price":15,"Title_Count":1},{"index":451,"State":"Ohio","Variety":"Cabernet Sauvignon","Avg_Points":84.0,"Avg_Price":12,"Title_Count":1},{"index":452,"State":"Ohio","Variety":"Chardonnay","Avg_Points":85.5,"Avg_Price":14,"Title_Count":2},{"index":453,"State":"Ohio","Variety":"Nebbiolo","Avg_Points":80.0,"Avg_Price":15,"Title_Count":1},{"index":454,"State":"Ohio","Variety":"Riesling","Avg_Points":84.43,"Avg_Price":14,"Title_Count":7}]'

In [12]:
st=list(df['State'].unique())

In [15]:
st

['America',
 'Arizona',
 'California',
 'Colorado',
 'Connecticut',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Iowa',
 'Kentucky',
 'Massachusetts',
 'Michigan',
 'Missouri',
 'Nevada',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'Ohio',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'Texas',
 'Vermont',
 'Virginia',
 'Washington',
 'Washington-Oregon']

In [16]:
state_avg=pd.read_sql('prpt',engine)
stateData=state_avg[(state_avg['State']==st)]#.to_json(orient='records')

ValueError: Arrays were different lengths: 748 vs 27

In [45]:
stateData['State'].values.tolist()

['Ohio', 'Ohio', 'Ohio', 'Ohio', 'Ohio']

In [46]:
data = [{
    "otu_ids": stateData['Avg_Points'].values.tolist(),
    "sample_values": stateData['Avg_Price'].values.tolist()
}]

In [47]:
data[0]['otu_ids']

[83.0, 84.0, 85.5, 80.0, 84.43]

In [31]:
engine = create_engine('sqlite:///../wine_reviews.sqlite')
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['reviews']

In [32]:
first_row=session.query(reviews).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x2b92f3424e0>,
 'country': 'US',
 'description': 'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.',
 'designation': None,
 'id': 1,
 'points': 87.0,
 'price': 14.0,
 'province': 'Oregon',
 'region_1': 'Willamette Valley',
 'region_2': 'Willamette Valley',
 'taster_name': 'Paul Gregutt',
 'taster_twitter_handle': '@paulgwine\xa0',
 'title': 'Rainstorm 2013 Pinot Gris (Willamette Valley)',
 'variety': 'Pinot Gris',
 'winery': 'Rainstorm'}

In [38]:
state=[]
variety=[]
points=[]
price=[]
title_cnt=[]

for row in session.query(reviews.province,reviews.variety,func.avg(reviews.points).label('Average_Point'),\
                         func.avg(reviews.price).label('Average_Price'),func.count(reviews.title).label('Title_cnt')).group_by(reviews.province,\
                        reviews.variety).order_by(reviews.province).all():
    state.append(row.province)
    variety.append(row.variety)
    title_cnt.append(row.Title_cnt)
    points.append(round(row.Average_Point,2))
    if (row.Average_Price != None):
        price.append(round(row.Average_Price))
    else:
        price.append(0)

In [39]:
engine = create_engine('sqlite:///../wine_reviews.sqlite')
state_prpt.to_sql('prpt', engine, if_exists='replace')

In [40]:
df=pd.read_sql('prpt',engine)
df.head()
st='Ohio'
df[(df['State']==st)]

,index,State,Variety,Avg_Points,Avg_Price,Title_Count
450,450,Ohio,Cabernet Franc,83.00,15,1
451,451,Ohio,Cabernet Sauvignon,84.00,12,1
452,452,Ohio,Chardonnay,85.50,14,2
453,453,Ohio,Nebbiolo,80.00,15,1
454,454,Ohio,Riesling,84.43,14,7


In [41]:
state_avg=pd.read_sql('prpt',engine)
states=list(state_avg['State'].unique())

In [42]:
states

['America',
 'Arizona',
 'California',
 'Colorado',
 'Connecticut',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Iowa',
 'Kentucky',
 'Massachusetts',
 'Michigan',
 'Missouri',
 'Nevada',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'Ohio',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'Texas',
 'Vermont',
 'Virginia',
 'Washington',
 'Washington-Oregon']